In [1]:
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv("/content/diabetes.csv")

In [4]:
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [7]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [9]:
# Separate features (X) and target (y)
# X contains the diagnostic measurements, y contains the diabetes outcome (0 or 1)
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [10]:
# SCALE THE DATA
# Deep Learning models converge faster and perform better when input features
# are on the same scale (mean=0, std=1). This prevents features with large values
# (like Insulin) from dominating features with small values (like DiabetesPedigreeFunction).
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [11]:
X=scaler.fit_transform(x)

In [12]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [15]:
X.shape

(768, 8)

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [86]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [19]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer="Adam",loss='binary_crossentropy',metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7559 - loss: 0.4774 - val_accuracy: 0.8052 - val_loss: 0.4659
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7755 - loss: 0.4637 - val_accuracy: 0.8052 - val_loss: 0.4664
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7649 - loss: 0.4854 - val_accuracy: 0.7987 - val_loss: 0.4674
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7690 - loss: 0.4799 - val_accuracy: 0.7792 - val_loss: 0.4672
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7952 - loss: 0.4354 - val_accuracy: 0.7857 - val_loss: 0.4666
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7894 - loss: 0.4486 - val_accuracy: 0.7857 - val_loss: 0.4685
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7793 - loss: 0.4548 - val_accuracy: 0.7792 - val_loss: 0.4682
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7827 - loss: 0.4469 - val_accuracy: 0.7792

In [23]:
#pip install -U keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.8 MB/s eta 0:00:00


In [25]:
import keras_tuner as kt

In [27]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [33]:
#creating tuner object
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [35]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [37]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [38]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#we are initializing epochs=6 as we already run 5 epochs
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

In [48]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)  #(start,end,steps)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [67]:
#tuner object
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name="pro")

Reloading Tuner from mydir/pro/tuner0.json


In [54]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 13s


In [56]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [57]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [59]:
model.fit(x_train,y_train,batch_size=100,initial_epoch=6,validation_data=(x_test,y_test))

In [68]:
#Select No. of layers
def build_model(hp):
  model=Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [69]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='numlayer')


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [70]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 10s


In [71]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [72]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [73]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7778 - loss: 0.4564 - val_accuracy: 0.7857 - val_loss: 0.4906
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7909 - loss: 0.4310 - val_accuracy: 0.7792 - val_loss: 0.4988
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7889 - loss: 0.4387 - val_accuracy: 0.7727 - val_loss: 0.4974
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8211 - loss: 0.4001 - val_accuracy: 0.7532 - val_loss: 0.5026
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8228 - loss: 0.4162 - val_accuracy: 0.7662 - val_loss: 0.5014
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7949 - loss: 0.4173 - val_accuracy: 0.7727 - val_loss: 0.5049
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8087 - loss: 0.4317 - val_accuracy: 0.7727 - val_loss: 0.5085
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8138 - loss: 0.4063 - val_accuracy: 0.77

In [ ]:
#

In [87]:
def build_model(hp):
  model=Sequential()
  counter=0
   # HYPERPARAMETER TUNING: LAYERS
    # We loop from 1 to 10 to let the tuner decide the optimal number of hidden layers.
    # This prevents us from manually guessing the depth of the network.
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
        # HYPERPARAMETER TUNING: NEURONS
        # In each layer, we test a range of neurons between 8 and 128.
        # We also test different activation functions ('relu' vs 'tanh').
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                     activation=hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh']),
                     input_dim=8))
      # REGULARIZATION
        # Randomly drop neurons during training to prevent overfitting.
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
         model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                     activation=hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh']),
                     ))
    counter+=1
# Output layer: 1 unit with sigmoid activation for Binary Classification (0 or 1)
    model.add(Dense(1,activation='sigmoid'))
    # HYPERPARAMETER TUNING: OPTIMIZER
    # We let the tuner choose the best optimizer (Adam is usually best, but we test others).
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
    loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [88]:
# INITIALIZE TUNER
# We use RandomSearch to test random combinations of hyperparameters.
# 'objective' tells the tuner to prioritize models with the highest Validation Accuracy.
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='completeDropout')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [89]:
# RUN THE SEARCH
# This triggers the training process. The tuner will build 5 models,
# train them for 5 epochs each, and track which one performs best.
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7142857313156128
Total elapsed time: 00h 00m 09s


In [91]:
tuner.get_best_hyperparameters()

In [92]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 8,
 'activation0': 'tanh',
 'dropout0': 0.9,
 'optimizer': 'sgd'}

In [94]:
model=tuner.get_best_models(num_models=1)[0]

In [95]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6282 - loss: 0.7642 - val_accuracy: 0.7143 - val_loss: 0.5912
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5916 - loss: 0.7747 - val_accuracy: 0.7273 - val_loss: 0.5870
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5997 - loss: 0.7660 - val_accuracy: 0.7208 - val_loss: 0.5842
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6663 - loss: 0.6869 - val_accuracy: 0.7143 - val_loss: 0.5815
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6655 - loss: 0.6804 - val_accuracy: 0.7143 - val_loss: 0.5777
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6460 - loss: 0.6843 - val_accuracy: 0.7338 - val_loss: 0.5745
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6486 - loss: 0.7462 - val_accuracy: 0.7338 - val_loss: 0.5735
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6515 - loss: 0.7203 - val_accuracy: 0.7

In [96]:
model.evaluate

<bound method TensorFlowTrainer.evaluate of <Sequential name=sequential, built=True>>